# Kleisli Categories

_P/42 Writer<T> c++ template_

we have accomplished our goal: the aggregation of the log
is no longer the concern of the individual functions

they produce their own messages, which are then, externally
concatenated into a larger log

we have to **abstract function composition**
   
   

## The Writer Category

MY NOTE:

what are in the writer category? a bunch of objects called "Writers" and their morphisms:

$Writer \rightarrow Writer'$



_P/45 composing Writer<T> template: a category point of view_
    
what is the identity morphism (in the Writer category) ?
    
an identity morphism (in the case of `Writer<T>`) should
pass its argument without change, and only contribute an
empty string to the log

**the only requirement is that the `log` type have monoidal
properties**

In [34]:
import Control.Monad
import Data.Char
import Data.String

-- the object
type Writer a = (a, String)

-- the morphism
f :: a -> Writer b
f = undefined

-- the new compose operator, the "fish" operator
(>=>) :: (a -> Writer b) -> (b -> Writer c) -> (a -> Writer c)
m1 >=> m2 = \x -> let (o1, state1) = m1 x
                      (o2, state2) = m2 o1
                  in (o2, state1 <> state2)

(>>=) :: Writer a -> (a -> Writer b) -> Writer b
(x1, s1) >>= f = let (x2, s2) = f x1
                 in (x2, s1 ++ s2)

return :: a -> Writer a
return x = (x, "called identity morphism\n")

-- chapter 20 formally introduce the `Kleisli arrow`
tell :: String -> Writer ()
tell s = ((), s)

add :: Int -> Int -> Writer Int
add n x = (n + x, "called add with n\n")

isEven :: Int -> Writer Bool
isEven x = (even x, "called isEven\n")

upCase :: String -> Writer String
upCase s = (fmap toUpper s, "called upCase\n")

toWords :: String -> Writer [String]
toWords s = (words s, "called toWords\n")

runTests :: IO ()
runTests = do
    let f = return >=> add 10 >=> isEven
    let (_, log1) = f 1
    mapM_ print $ lines log1
    
    print "---------------------------------------"
    
    let g = return >=> upCase >=> toWords
    let (_, log2) = g "iddqd idkfa idnoclip"
    mapM_ print $ lines log2
    
    print "---------------------------------------"
    
    let t = return 
         >=> (\x -> tell "telling something\n" >>= const (upCase x))
    let (_, log3) = t "test tell"
    mapM_ print $ lines log3
runTests

"called identity morphism"
"called add with n"
"called isEven"
"---------------------------------------"
"called identity morphism"
"called upCase"
"called toWords"
"---------------------------------------"
"called identity morphism"
"telling something"
"called upCase"

_P/49 what is a Kleisli category_

a Kleisli category has, as objects, **the types of the underlying
programming language**

morphisms from type $A$ to type $B$ are fujnctions that go
from $A$ to a type derived from $B$ using the particular
embellishment

each Kleisli category defines its own way of composing such
morphisms, as well as the identity morphisms with respect
to that composition

a category where morphisms are represented by embellished 
functions, and their composition does more than just pass
the output of one function to the input of another

**we have more degree of freedom to play with: the composition
itself**

## Challenge

In [32]:
data Optional a
  = Value a
  | Nothing
  deriving (Eq, Show)
  
(>=>) :: (a -> Optional b) -> (b -> Optional c) -> (a -> Optional c)
(>=>) fa fb x = case fa x of
                    Value oa -> fb oa
                    Nothing -> Nothing
                        
safeReciprocal :: Float -> Optional Float
safeReciprocal x = if x >= 0 then Value $ sqrt x else Nothing

safeRoot :: Float -> Optional Float
safeRoot x = if x /= 0 then Value $ 1 / x else Nothing

runTests :: IO ()
runTests = do
    let f = safeReciprocal >=> safeRoot
        good = f 12.1
        bad = f 0.0
    print good
    print bad

runTests

Value 0.2874798
Nothing